## Importing Important Libraries
### Steps To Be Followed

* Importing necessary Libraries
* Creating S3 bucket
* Mapping train And Test Data in S3
* Mapping The path of the models in S3

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session, TrainingInput

In [2]:
## boto3 session
boto3_session = boto3.session.Session()
boto3_session

Session(region_name='ap-south-1')

In [3]:
## getting the region of sagemaker notebook instance
region_name = boto3_session.region_name
region_name

'ap-south-1'

In [4]:
## s3 bucket selection for artifact storage
## creating the s3 bucket or selecting an already created s3 bucket in the same AWS region
s3_bucket_name = 'mlflow-artifacts-raj'

In [5]:
## listing available client resources
s3_client = boto3.client('s3')
bucket_resources = s3_client.list_buckets()
list_avail_buckets = [b['Name'] for b in bucket_resources['Buckets']]
list_avail_buckets

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


['demo-raj-bucket-2022-1',
 'demo-raj-cors-bucket-2022-1',
 'mlflow-artifacts-raj']

In [6]:
## selecting/creating the bucket for sagemaker needs
s3_resource = boto3.resource('s3')

if s3_bucket_name in list_avail_buckets:
    asm_bucket = s3_resource.Bucket(s3_bucket_name)
    print(f'Alreay present Bucket Name: {asm_bucket.name}, Creation Date: {asm_bucket.creation_date}')
    bucket_name = asm_bucket.name
else:
    #creating the bucket:
    try:
        if  my_region == region_name:
            s3.create_bucket(Bucket=s3_bucket_name)
            bucket_name = s3_bucket_name
            print(f'S3 bucket {s3_bucket_name} created successfully')
        else:
            print('Please check your aws region.')
            bucket_name = None
    except Exception as e:
        print('S3 error: ',e)


Alreay present Bucket Name: mlflow-artifacts-raj, Creation Date: 2022-06-04 06:48:31+00:00


In [7]:
print(bucket_name)

mlflow-artifacts-raj


In [8]:
## set an output path in s3 bucket where the trained model will be stored
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://mlflow-artifacts-raj/xgboost-as-a-built-in-algo/output


In [9]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [10]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [11]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [12]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [13]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [16]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [18]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
